# Mapping Taylor Swift concerts

### Taylor Swift concert dates:

<b> June 2024

Edinburgh (Murrayfield, EH12 5PJ, 67,144) x3

Liverpool (Anfield, L4 0TH, 61,276) x3

Cardiff (Principality,  CF10 1NS, 74,500) x1

London (Wembley, HA9 0WS, 90,000) x3

<b> August 2024

London (Wembley, HA9 0WS, 90,000) x3 

In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_articles/src/utils')

from google.cloud import bigquery
import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns


import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import plotly.colors as pc


from fintrans_toolbox.src import table_utils as t

from google.cloud import storage
import geopandas as gpd
import ipywidgets as widgets
from ipywidgets import interactive
import folium
from IPython.display import display
from fintrans_toolbox.src import bq_utils as bq

client = bigquery.Client()

# Plotting inbound tourism spending at stadium areas

### Loading Visa data

Concert districts: EH12, L4, CF10, HA9

In [ ]:
sql_spend = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
merchant_location_level != 'All' AND
cardholder_issuing_level != 'All' AND
merchant_location_level = 'POSTAL_DISTRICT'
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

district_df = client.query(sql_spend).to_dataframe()
district_df = t.create_date_time(district_df)

sml_df1 = district_df[(district_df['mcc'] != 'All')].copy()
sml_df2 = district_df[(district_df['mcg'] == 'All')].copy()

df = pd.concat([sml_df1, sml_df2])

In [ ]:
# Filtering to retain only concert area districts
df = df[df['merchant_location'].str.match(r'^(EH|L|CF|HA)\d+')].copy()

### Loading geo data

In [ ]:
# client = storage.Client()
# bq.boundary_file_download(client, postal_level = "postcode_district", output_location = '')

district_shape_full = gpd.read_file('postcode_district.shp')
# Retaining only concert area districts
district_shape = district_shape_full[district_shape_full['postdistri'].isin(df['merchant_location'].unique())].copy()
eh_shape = district_shape[district_shape['postdistri'].str.startswith(('EH'))].copy()
l_shape = district_shape[district_shape['postdistri'].str.startswith(('L'))].copy()
ha_shape = district_shape[district_shape['postdistri'].str.startswith(('HA'))].copy()
cf_shape = district_shape[district_shape['postdistri'].str.startswith(('CF'))].copy()

## Calculating metrics

In [ ]:
def calc_index_yoy(df, need_date_cols, group_list):
    
    result = df.copy()
    if need_date_cols is True:
        result['year'] = result['date_time'].dt.year
        result['month'] = result['date_time'].dt.month
    
    metrics = ['spend', 'transactions', 'cardholders']
    month_group = group_list + ['month']

    for i in metrics:
        # calc year-on-year differences
        result[f'yoy_{i}'] = result.groupby(month_group)[f'{i}'].diff(periods=1)

        # calc year-on-year % change
        result[f'yoy_{i}_perc'] = result.groupby(month_group)[f'{i}'].pct_change(periods=1)*100

        # index to 2019 average
        result[f'index_{i}_2019'] = result.groupby(group_list)[f'{i}'].transform(lambda x: x / (x.iloc[0:11].mean(axis = 0))*100)
        
    return result

e.g. Edinburgh

In [ ]:
eh_df = df[(df['merchant_location'].str.startswith('EH')) &
          (~df['cardholder_issuing_country'].isin(['All']))].copy()

In [ ]:
eh_df2 = calc_index_yoy(df = eh_df, 
                              need_date_cols = True, 
                              group_list = ['merchant_location', 
                                            'cardholder_issuing_country',
                                            'mcg'])

In [ ]:
eh_geo = eh_shape.merge(eh_df2, how = 'left', left_on = 'postdistri', right_on = 'merchant_location')

------------------------------

In [ ]:
def plot_interactive_map(gdf):
    def plot_map(tod, country, metric):
        max_val = gdf[metric].max()
        min_val = gdf[metric].min()
        
        # Filter GeoDataFrame based on the selected filters
        filtered_gdf = gdf[(gdf['time_period_value'] == tod) & (gdf['cardholder_issuing_country'] == country)]
        
        # Ensure there is data to plot
        if filtered_gdf.empty:
            print("No data for the selected filters")
            return
         
        map_object = filtered_gdf.explore(column=metric, 
                                          tooltip=['postdistri', metric], 
                                          legend=True,
                                         #vmin = min_val,
                                         #vmax = max_val
                                         )
        
        display(map_object)
    
    # Create dropdown widgets for date, country, and metric
    tod = list(gdf['time_period_value'].unique())
    metrics = list(gdf.columns[11:14]) + list(gdf.columns[16:])
    cc = list(gdf['cardholder_issuing_country'].unique())

    # Create the dropdown widgets
    date_dropdown = widgets.Dropdown(options=tod, description="Date:")
    country_dropdown = widgets.Dropdown(options=cc, description="Card Country:")
    metric_dropdown = widgets.Dropdown(options=metrics, description="Metric:")

    # Use widgets.interactive to create the interactive widget
    return interactive(plot_map, tod=date_dropdown, country=country_dropdown, metric=metric_dropdown)


In [ ]:
#plot_interactive_map(eh_geo)

-------

# Plotting domestic movements 

### Loading Visa data

In [ ]:
client = bigquery.Client()

sql_rphst = """SELECT time_period_value, merchant_location, cardholder_location, mcg, spend, transactions, cardholders
  FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
  WHERE cardholder_location_level = 'POSTAL_DISTRICT' AND
  merchant_location_level = 'POSTAL_DISTRICT' AND time_period = 'Month'
  AND merchant_location IN ('EH12', 'L4', 'CF10', 'HA9') 
  AND cardholder_location != 'UNKNOWN'
  ORDER BY time_period, time_period_value,spend"""
district_ret = client.query(sql_rphst).to_dataframe()
district_ret = t.create_date_time(district_ret)

In [ ]:
# Calculating the proportion of spend that each cardholder location makes up of total merchant location spend
district_ret['dis_spend'] = district_ret.groupby(['date_time','merchant_location', 'mcg'])['spend'].transform('sum')
district_ret['perc_dis_spend'] = district_ret['spend']/district_ret['dis_spend']*100

### Merging geodata to each cardholder locations


In [ ]:
retail_geo = district_shape_full.merge(district_ret, how = 'right', left_on = 'postdistri', right_on = 'cardholder_location')

In [ ]:
def plot_domestic_map(gdf, stadium_district):
    def plot_dom_map(tod, mcg):
        
        # Filter GeoDataFrame based on the selected filters
        filtered_gdf = gdf[(gdf['time_period_value'] == tod) & (gdf['merchant_location'] == stadium_district) & (gdf['mcg'] == mcg)]
    
        
        # Ensure there is data to plot
        if filtered_gdf.empty:
            print("No data for the selected filters")
            return
         
        
        map_object = filtered_gdf.explore(column='perc_dis_spend', 
                                          tooltip=['postdistri', 'perc_dis_spend', 'mcg', 'date_time'], 
                                          legend=True,
                                          height = 600,
                                          width = 600
                                         )
        
        
        
        display(map_object)
    
    # Create dropdown widgets for date, country, and metric
    tod = list(gdf['time_period_value'].unique())
    mcgl = list(gdf['mcg'].unique())

    # Create the dropdown widgets
    date_dropdown = widgets.Dropdown(options=tod, description="Date:")
    mcg_dropdown = widgets.Dropdown(options=mcgl, description="MCG:")

    # Use widgets.interactive to create the interactive widget
    return interactive(plot_dom_map, tod=date_dropdown, mcg = mcg_dropdown)

### If slow :

In [ ]:
def plot_domestic_map_simple(gdf, stadium_district):
    def plot_dom_map(tod1, tod2, mcg1, mcg2):
        
        fig, ((ax1,ax2)) = plt.subplots(1,2,sharey = True,figsize = (10,13))

        gdf[(gdf['time_period_value']==tod1) &
           (gdf['mcg'] ==mcg1) &
           (gdf['merchant_location'] == stadium_district)].plot(ax=ax1, 
                                                 column='perc_dis_spend', 
                                                 legend=True, 
                                                 legend_kwds={'shrink': 0.3}, 
                                                 cmap="viridis")
        
        gdf[(gdf['time_period_value']==tod2) &
           (gdf['mcg'] == mcg2)&
           (gdf['merchant_location'] == stadium_district)].plot(ax=ax2, 
                                                 column='perc_dis_spend',
                                                 legend=True, 
                                                 legend_kwds={'shrink': 0.3},
                                                 cmap="viridis")
        
        ax1.set_title(f'{stadium_district}, mcg = {mcg1} {tod1}')
        ax2.set_title(f'{stadium_district}, mcg = {mcg2} {tod2}')



        plt.show()
    
    # Create dropdown widgets for date, country, and metric
    tod1 = list(gdf['time_period_value'].unique())
    mcgl1 = list(gdf['mcg'].unique())
    
    tod2 = list(gdf['time_period_value'].unique())
    mcgl2 = list(gdf['mcg'].unique())

    # Create the dropdown widgets
    date_dropdown1 = widgets.Dropdown(options=tod1, description="Date:")
    mcg_dropdown1 = widgets.Dropdown(options=mcgl1, description="MCG:")
    
    date_dropdown2 = widgets.Dropdown(options=tod2, description="Date:")
    mcg_dropdown2 = widgets.Dropdown(options=mcgl2, description="MCG:")

    # Use widgets.interactive to create the interactive widget
    return interactive(plot_dom_map, tod1=date_dropdown1, mcg1 = mcg_dropdown1, tod2 = date_dropdown2, mcg2 = mcg_dropdown2)

In [ ]:
plot_domestic_map_simple(retail_geo, stadium_district = 'EH12')